# REENTRENAMIENTO TYR - GUARDAR PESOS Y MODELO COMPLETO

Este notebook entrena el modelo y guarda tanto los pesos (.pth) como el modelo completo (.safetensors).

**IMPORTANTE**: Verifica que el modelo funcione ANTES de guardarlo (PASO 12)

## PASO 1: Instalar dependencias

In [ ]:
!pip install transformers torch datasets scikit-learn -q

## PASO 2: Importar librerías

In [ ]:
import json
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    Trainer, 
    TrainingArguments
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

print("Librerías importadas correctamente")
print(f"Torch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

## PASO 3: Cargar Dataset

**INSTRUCCIONES:**
1. Sube el archivo `Dataset_TYR_3000_FINAL.json` a Colab
2. Ejecuta esta celda

In [ ]:
# Cargar dataset
with open('Dataset_TYR_3000_FINAL.json', 'r', encoding='utf-8') as f:
    data_raw = json.load(f)

print(f"Dataset cargado: {len(data_raw)} ejemplos")

# Contar por intención
intenciones_count = {}
for texto, intencion in data_raw:
    intenciones_count[intencion] = intenciones_count.get(intencion, 0) + 1

print("\nDistribución por intención:")
for intencion, count in sorted(intenciones_count.items()):
    print(f"  {intencion}: {count}")

## PASO 4: Crear label_map

In [ ]:
# Label map
label_map = {
    "becas_financiamiento": 0,
    "contacto_ubicacion": 1,
    "faq_general": 2,
    "fuera_dominio": 3,
    "horarios_duracion": 4,
    "informacion_carreras": 5,
    "informacion_institucional": 6,  # NUEVA CLASE
    "inscripcion_admision": 7,
    "requisitos_ingreso": 8,
    "saludo_despedida": 9
}

id2label = {v: k for k, v in label_map.items()}

print(f"Label map creado: {len(label_map)} clases")
print("\nClases:")
for idx, label in sorted(id2label.items()):
    print(f"  {idx}: {label}")

## PASO 5: Preparar datos

In [ ]:
# Convertir a formato con IDs numéricos
textos = [item[0] for item in data_raw]
labels = [label_map[item[1]] for item in data_raw]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    textos, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Train: {len(X_train)} ejemplos")
print(f"Test: {len(X_test)} ejemplos")

# Verificar que informacion_institucional está presente
info_inst_train = sum(1 for label in y_train if label == 6)
info_inst_test = sum(1 for label in y_test if label == 6)
print(f"\nEjemplos de 'informacion_institucional' (clase 6):")
print(f"  Train: {info_inst_train}")
print(f"  Test: {info_inst_test}")

## PASO 6: Cargar tokenizer

In [ ]:
model_name = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Tokenizer cargado: {model_name}")

## PASO 7: Tokenizar datos

In [ ]:
# Crear datasets
train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
test_dataset = Dataset.from_dict({"text": X_test, "label": y_test})

# Tokenizar
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

print("Datos tokenizados")

## PASO 8: Cargar modelo BERT base

In [ ]:
# Cargar modelo base con 10 clases
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_map),
    id2label=id2label,
    label2id=label_map,
    ignore_mismatched_sizes=True
)

print(f"Modelo cargado con {len(label_map)} clases")
print(f"Classifier output size: {model.classifier.out_features}")

## PASO 9: Configurar entrenamiento

In [ ]:
# Métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

# Training arguments - NO GUARDAR CHECKPOINTS
training_args = TrainingArguments(
    output_dir="./results_temp",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="no",  # NO GUARDAR NADA durante el training
    logging_dir="./logs",
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    warmup_steps=500,
    report_to="none"
)

# Trainer SIN EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

print("Trainer configurado (sin early stopping)")

## PASO 10: ENTRENAR

In [ ]:
print("Iniciando entrenamiento...\n")
trainer.train()
print("\nEntrenamiento completado")

## PASO 11: Evaluar en test set

In [ ]:
# Evaluar
results = trainer.evaluate()
print("\n" + "=" * 60)
print("RESULTADOS EN TEST SET")
print("=" * 60)
print(f"Accuracy: {results['eval_accuracy']:.4f} ({results['eval_accuracy']*100:.2f}%)")
print(f"F1-Score: {results['eval_f1']:.4f} ({results['eval_f1']*100:.2f}%)")
print("=" * 60)

## PASO 12: PROBAR MODELO EN MEMORIA (ANTES DE GUARDAR)

**CRÍTICO**: Probar el modelo ANTES de guardarlo para verificar que funciona correctamente

In [ ]:
# Preguntas de prueba sobre información institucional
test_questions = [
    "Cuándo se fundó el ITSE?",
    "Qué reconocimientos tiene el ITSE?",
    "El MIT colabora con el ITSE?",
    "Cuál es la empleabilidad del ITSE?",
    "Qué es el CAIPI?",
    "Quién es la rectora del ITSE?"
]

# Poner modelo en modo evaluación
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("\n" + "=" * 60)
print("PRUEBA DE PREGUNTAS INSTITUCIONALES (MODELO EN MEMORIA)")
print("=" * 60)

correctas = 0
for i, question in enumerate(test_questions, 1):
    # Tokenizar
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Predecir
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        pred_class = outputs.logits.argmax(-1).item()
        confidence = probs[0][pred_class].item()
    
    pred_label = id2label[pred_class]
    is_correct = pred_label == "informacion_institucional"
    if is_correct:
        correctas += 1
    
    status = "✅" if is_correct else "❌"
    print(f"{i}. {status} \"{question}\"")
    print(f"   → {pred_label} ({confidence*100:.2f}%)\n")

print("=" * 60)
print(f"RESULTADO: {correctas}/6 correctas")
print("=" * 60)

if correctas >= 5:
    print("\n✅ MODELO FUNCIONA CORRECTAMENTE - Continuar con guardado")
else:
    print("\n⚠️ ADVERTENCIA: El modelo no clasifica correctamente")
    print("NO continuar con el guardado")

## PASO 13: GUARDAR PESOS (.pth)

**IMPORTANTE**: Solo ejecutar si el PASO 12 muestra 5/6 o 6/6 correctas

In [ ]:
# Guardar SOLO los pesos del modelo
torch.save(model.state_dict(), 'modelo_tyr_10_clases_PESOS.pth')

print("\n" + "=" * 60)
print("PESOS GUARDADOS EN: modelo_tyr_10_clases_PESOS.pth")
print("=" * 60)

## PASO 14: GUARDAR MODELO COMPLETO (.safetensors)

Guardamos también el modelo completo para tener ambos formatos

In [ ]:
import os

# Crear carpeta para el modelo completo
modelo_dir = "modelo_bert_tyr_10_clases_COMPLETO"
os.makedirs(modelo_dir, exist_ok=True)

# Guardar modelo completo
model.save_pretrained(modelo_dir)
tokenizer.save_pretrained(modelo_dir)

# Guardar label_map
label_map_save = {str(k): v for k, v in id2label.items()}
with open(f'{modelo_dir}/label_map.json', 'w', encoding='utf-8') as f:
    json.dump(label_map_save, f, ensure_ascii=False, indent=2)

print("\n" + "=" * 60)
print(f"MODELO COMPLETO GUARDADO EN: {modelo_dir}/")
print("=" * 60)
print("\nArchivos guardados:")
print("  - config.json")
print("  - model.safetensors")
print("  - tokenizer_config.json")
print("  - vocab.txt")
print("  - label_map.json")

## PASO 15: VERIFICAR modelo completo guardado

Cargar el modelo desde la carpeta y verificar que funciona

In [ ]:
# Cargar modelo desde carpeta
model_verificacion = AutoModelForSequenceClassification.from_pretrained(modelo_dir)
model_verificacion.eval()
model_verificacion.to(device)

print("\n" + "=" * 60)
print("VERIFICACIÓN: MODELO COMPLETO DESDE CARPETA")
print("=" * 60)

correctas_completo = 0
for i, question in enumerate(test_questions, 1):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model_verificacion(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        pred_class = outputs.logits.argmax(-1).item()
        confidence = probs[0][pred_class].item()
    
    pred_label = id2label[pred_class]
    is_correct = pred_label == "informacion_institucional"
    if is_correct:
        correctas_completo += 1
    
    status = "✅" if is_correct else "❌"
    print(f"{i}. {status} \"{question}\"")
    print(f"   → {pred_label} ({confidence*100:.2f}%)\n")

print("=" * 60)
print(f"RESULTADO: {correctas_completo}/6 correctas")
print("=" * 60)

if correctas_completo >= 5:
    print("\n✅ VERIFICACIÓN EXITOSA - Modelo completo funciona")
else:
    print("\n❌ ERROR: El modelo completo NO funciona correctamente")

## PASO 16: Comprimir modelo completo

Crear un ZIP del modelo completo para descarga más fácil

In [ ]:
import shutil

# Crear ZIP
zip_filename = "modelo_bert_tyr_10_clases_COMPLETO"
shutil.make_archive(zip_filename, 'zip', modelo_dir)

print("\n" + "=" * 60)
print(f"ZIP CREADO: {zip_filename}.zip")
print("=" * 60)

# Mostrar tamaño del archivo
import os
size_mb = os.path.getsize(f"{zip_filename}.zip") / (1024 * 1024)
print(f"\nTamaño: {size_mb:.2f} MB")

## PASO 17: RESUMEN FINAL

Lista de archivos para descargar

In [ ]:
print("\n" + "=" * 60)
print("ARCHIVOS LISTOS PARA DESCARGAR")
print("=" * 60)
print("\n1. modelo_tyr_10_clases_PESOS.pth (~420 MB)")
print("   - Solo pesos del modelo")
print("   - Usar con script cargar_pesos_nuevo_modelo.py")
print("\n2. modelo_bert_tyr_10_clases_COMPLETO.zip (~400-500 MB)")
print("   - Modelo completo con safetensors")
print("   - Listo para usar directamente")
print("   - Incluye: model.safetensors, config.json, tokenizer, label_map.json")
print("\n" + "=" * 60)
print("RECOMENDACIÓN: Descarga AMBOS archivos")
print("=" * 60)
print("\nVerificaciones realizadas:")
print(f"  - Modelo en memoria: {correctas}/6 correctas ✅" if correctas >= 5 else f"  - Modelo en memoria: {correctas}/6 correctas ❌")
print(f"  - Modelo completo guardado: {correctas_completo}/6 correctas ✅" if correctas_completo >= 5 else f"  - Modelo completo guardado: {correctas_completo}/6 correctas ❌")
print("\n✅ TODO LISTO PARA DESCARGAR")